In [1]:
import pandas as pd
import numpy as np
!pip install pyxlsb

## Bước 1. Đọc dữ liệu

In [2]:
## Đọc hết các sheet data trong file
dict_df = pd.read_excel('data_case_production_classroom.xlsb', sheet_name=None)
#Trả về dạng dictionary, với key là sheet_name, và value là sheet_content

In [3]:
## Prints all the sheets name
print(dict_df.keys())

dict_keys(['Guideline', 'CRD - 0', 'ORDER TRACKING - 1.1', 'MATERIAL REQUEST - 1.2', 'KITTING - 2', 'PRINTING - 3', 'BCSX - 4', 'Inventory - 6'])


### NOTE

## Bước 2. Planning pending

In [4]:
### Planning pending = Order Tracking - Material Request  => LEFT ANTI JOIN
# Sheetname: Order Tracking = ORDER TRACKING - 1.1
# Sheetname: Material Request = MATERIAL REQUEST - 1.2

In [5]:
## get dataframe from dict
df_OrderTracking = dict_df .get('ORDER TRACKING - 1.1')
df_MaterialRequest = dict_df .get('MATERIAL REQUEST - 1.2')

In [6]:
df_planning = pd.merge(df_OrderTracking, df_MaterialRequest, \
                               on=['SO#','SO#'], how="outer", indicator=True)
df_planning

#df_planning = pd.merge(dict_df .get('ORDER TRACKING - 1.1'), dict_df .get('MATERIAL REQUEST - 1.2'), \
                       # on=['SO#','SO#'], how="outer", indicator=True)

,Ngày,SO#,QTY,Cell In,_merge
0,44690.0,79149244-1,300.0,NaN,both
1,44690.0,79155892-1,70.0,NaN,both
2,44690.0,79155892-2,70.0,NaN,both
3,44690.0,79164445-1,200.0,NaN,both
4,44690.0,79164445-2,300.0,NaN,both
...,...,...,...,...,...
81053,NaN,78994383-2,NaN,NaN,right_only
81054,NaN,78994383-3,NaN,NaN,right_only
81055,NaN,79071486-1,NaN,NaN,right_only
81056,NaN,78359140-1,NaN,NaN,right_only


In [34]:
# Từ df_planning, lấy dữ liệu mà tại đó cột _merge có giá trị 'left_only' (left_only - nghĩa là dòng dữ liệu SO# đó chỉ nằm ở Left table - Order Tracking)
# Trả dữ liệu - chỉ lấy cột SO#

df_planning_pending = df_planning.loc[df_planning['_merge'] == 'left_only', ['SO#']]

#Cách khác:
#df_planning_pending = df_planning[df_planning['_merge'] == 'left_only'] [['SO#']]
#Step by step
#df_planning_pending = df_planning[df_planning['_merge'] == 'left_only'] 
#df_planning_pending = df_planning_pending [['SO#']]

df_planning_pending [['Cell In','Status Order']] = ['','PLANNING']
df_planning_pending

,SO#,Cell In,Status Order
197,79173263-1,,PLANNING
209,79163865-1,,PLANNING
210,79163865-2,,PLANNING
211,79163865-3,,PLANNING
212,79163865-4,,PLANNING
...,...,...,...
8012,79419520-8,,PLANNING
8013,79419520-9,,PLANNING
8014,79419520-10,,PLANNING
8015,79419710-1,,PLANNING


## Bước 3. Kitting pending

In [8]:
### Kitting pending = Material Request + (Kitting LEFT ANTI JOIN Printing)
# Sheetname: Material Request = MATERIAL REQUEST - 1.2
# Sheetname: Kitting = KITTING - 2
# Sheetname: Printing = PRINTING - 3

#### 3.1 Kitting & Printing

In [9]:
# Vì Printing không có cột "Cell In" nên sẽ Join với Kitting để lấy Cell In (mục đích lấy Cell In: nếu Cell In = null -> IPPS -> Status order = 'Packing')
# "Cell In" đến Kitting mới có dữ liệu
# Kiểu JOIN giữa Printing và Kitting: Printing LEFT Kitting 
# (hoặc sử dụng INNER JOIN vì có ở Printing thì đã có ở KITTING, nên LEFT hay INNER sẽ như nhau)

In [9]:
df_printing = dict_df .get('PRINTING - 3')
df_kitting = dict_df .get('KITTING - 2')

In [10]:
df_printing = pd.merge(df_printing, df_kitting, \
                       on=['SO#','SO#'], how="left", indicator=True) [['SO#', 'Cell In']]
df_printing

,SO#,Cell In
0,69033554-1,AS 1667
1,72879582-1,AS 306
2,73310043-1,AS 546
3,73310043-2,AS 547
4,73310052-1,AS 711
...,...,...
79433,79074031-2,NaN
79434,79063273-2,NaN
79435,79063273-1,NaN
79436,79074031-1,NaN


In [26]:
#Check unique SO#, Cell in
#Sử dụng Value_Counts, tuy nhiên nếu có nhiều dòng duplicate SO#-Cell in thì cần Drop Duplicate trước vì mục đích mình muốn check xem 1 SO# có bao nhiêu Cell in
df_printing.drop_duplicates(subset=['SO#', 'Cell In'], inplace = True)
df_printing['SO#'].value_counts().max()

1

In [38]:
### Kitting pending 1 = Kitting LEFT ANTI JOIN Printing (tương tự Planning Pending)

#Join lấy primary key là SO#, không có Cell In, nên để lấy Cell In ra, đặt Suffixes (như T1. hay T2. của SQL), default Suffix là _x và _y)

df_printing_kitting = pd.merge(df_kitting, df_printing, \
                       on=['SO#','SO#'], how="outer", suffixes=('_kitting', '_printing'), indicator=True)
df_kitting_pending_1 = df_printing_kitting.loc[df_printing_kitting['_merge'] == 'left_only', ['SO#', 'Cell In_kitting']]
df_kitting_pending_1.rename (columns={'Cell In_kitting': 'Cell In'}, inplace = True)
#df_kitting_pending_1 = df_kitting_pending_1.rename (columns={'Cell In_kitting': 'Cell In'})

df_kitting_pending_1

,SO#,Cell In
4270,75907156-1,AS 4271
13162,79300753-1,AS 13163
13163,79300753-3,AS 13164
13164,79300753-4,AS 13165
13165,79300753-5,AS 13166
...,...,...
79504,78901954-3,NaN
79505,78182553-1,NaN
79506,78192220-1,NaN
79507,79328976-1,NaN


#### 3.2 Kitting (Material Request)

In [39]:
### Kitting Pending 2 = Material Request (nếu SO# không có trong Kitting pending 1 thì sẽ không có ở những pending phía sau 
# => khi Union lại, chỉ cần xóa duplicate với Kitting pending 1)

df_kitting_pending_2 = df_MaterialRequest [['SO#', 'Cell In']]
df_kitting_pending_2

,SO#,Cell In
0,78181470-1,NaN
1,78181467-1,NaN
2,78175437-1,NaN
3,78175437-2,NaN
4,78175066-1,NaN
...,...,...
74500,78994383-2,NaN
74501,78994383-3,NaN
74502,79071486-1,NaN
74503,78359140-1,NaN


#### 3.3 Kitting Pending (final)

In [40]:
### Kitting Pending final
df_kitting_pending = pd.concat([df_kitting_pending_1, df_kitting_pending_2]).drop_duplicates(subset=['SO#', 'Cell In'])
df_kitting_pending ['Status Order'] = 'KITTING'
df_kitting_pending

,SO#,Cell In,Status Order
4270,75907156-1,AS 4271,KITTING
13162,79300753-1,AS 13163,KITTING
13163,79300753-3,AS 13164,KITTING
13164,79300753-4,AS 13165,KITTING
13165,79300753-5,AS 13166,KITTING
...,...,...,...
74500,78994383-2,NaN,KITTING
74501,78994383-3,NaN,KITTING
74502,79071486-1,NaN,KITTING
74503,78359140-1,NaN,KITTING


In [48]:
#Check unique SO#, Cell in 

df_kitting_pending.drop_duplicates(subset=['SO#'], inplace = True)
#Drop duplicate theo SO#, vì Material Request sẽ chưa có dữ liệu Cell in, nên Join lại sẽ NaN, drop duplicate theo [SO#, Cell in] sẽ giữ lại
df_kitting_pending['SO#'].value_counts().max()

1

## Bước 4. Packing & Printing pending

#### 4.1 Packing & Printing từ BCSX

In [18]:
### Packing pending: Dùng BCSX (Báo cáo sản xuất hàng ngày của công nhân) và CRD (order của khách hàng)
# Nếu BCSX.[Qty] >= CRD.[Qty] => Status Order = 'Packing' else 'Printing'   ['Qty' trong BCSX đã Group by SO#]
# BCSX: LEFT JOIN với KITTING lấy "Cell In" (hoặc sử dụng INNER JOIN vì có ở BCSX thì đã có ở KITTING, nên LEFT hay INNER sẽ như nhau)
# BCSX: LEFT JOIN với CRD lấy Qty so sánh (hoặc sử dụng INNER JOIN, vì có ở BCSX thì đã phải có ở CRD, nên LEFT hay INNER sẽ như nhau)
# Tuy nhiên, trong data hiện tại có những SO# không có trong CRD --> nên lấy INNER, xem như có Order trong CRD mới tính/track.
# Sheet name: BCSX = BCSX - 4
# Sheet name: CRD = CRD - 0
# Sheet name: KITTING = KITTING - 2

In [50]:
df_BCSX = dict_df .get('BCSX - 4')
df_CRD = dict_df .get('CRD - 0')

In [51]:
df_BCSX_Cell = pd.merge(df_BCSX, df_kitting, \
                       on=['SO#','SO#'], how="left", suffixes =('_BCSX', '_Kitting'), indicator=True)[['SO#', 'Cell In', 'QTY']]
df_BCSX_Cell.drop_duplicates(subset=['SO#', 'Cell In'], inplace = True)
df_BCSX_Cell

,SO#,Cell In,QTY
0,63371643-40,NaN,17300
1,63371643-41,NaN,1960
2,63371643-42,NaN,160
3,63371643-43,NaN,2936
4,63371643-44,NaN,210
...,...,...,...
187092,79329523-1,AS 72044,374
187093,79329523-2,AS 72043,82
187094,79329578-1,AS 13153,416
187095,79329907-1,AS 72045,150


In [52]:
df_BCSX_CRD = pd.merge(df_BCSX_Cell, df_CRD, \
                       on=['SO#','SO#'], how="inner", suffixes =('_BCSX', '_CRD')) [['SO#', 'Cell In', 'QTY_BCSX', 'QTY_CRD']]
df_BCSX_CRD ['Status Order'] = np.where (df_BCSX_CRD['QTY_BCSX'] >= df_BCSX_CRD['QTY_CRD'], 'PACKING', 'PRINTING')

#Code check lại những SO# status 'Printing'
#df_BCSX_CRD[df_BCSX_CRD ['Status Order'] == 'PRINTING']

df_BCSX_CRD

,SO#,Cell In,QTY_BCSX,QTY_CRD,Status Order
0,71324345-1,NaN,8757,8757.0,PACKING
1,73861695-1,NaN,7567,7567.0,PACKING
2,74955169-1,AS 2821,156,156.0,PACKING
3,74963701-1,AS 2826,445,445.0,PACKING
4,74966020-1,AS 2828,1627,1627.0,PACKING
...,...,...,...,...,...
9077,79329523-1,AS 72044,374,374.0,PACKING
9078,79329523-2,AS 72043,82,82.0,PACKING
9079,79329578-1,AS 13153,416,416.0,PACKING
9080,79329907-1,AS 72045,150,150.0,PACKING


In [53]:
#Check unique SO#, Cell in 

df_BCSX_CRD.drop_duplicates(subset=['SO#', 'Cell In', 'Status Order'], inplace = True)
df_BCSX_CRD['SO#'].value_counts().max()

1

#### 4.2 Printing từ DL Printing

In [55]:

### PRINTING pending (cont.)
# Có SO# trong Printing thì sẽ mang Status là "Printing", và BCSX sẽ không bao gồm hết các SO# có trong "Printing"
# Do đó, cần lấy những SO# Pending ở Printing mà chưa có trong BCSX. Nếu đã có trong BCSX, thì đã có logic để tạo "Status Order" => LEFT ANTI JOIN

df_printing_BCSX = pd.merge(df_printing, df_BCSX, \
                       on=['SO#','SO#'], how="outer", indicator=True)
     #df_printing đã lấy Cell In ở Bước 3
    
df_printing_pending = df_printing_BCSX.loc[df_printing_BCSX['_merge'] == 'left_only', ['SO#', 'Cell In']]
df_printing_pending ['Status Order'] = 'PRINTING'
df_printing_pending

,SO#,Cell In,Status Order
15,73367792-1,AS 8632,PRINTING
23,73655517-1,AS 8626,PRINTING
48,73872083-2,AS 8569,PRINTING
65,73978931-1,AS 8561,PRINTING
66,73978931-2,AS 8562,PRINTING
...,...,...,...
79418,79074031-2,NaN,PRINTING
79419,79063273-2,NaN,PRINTING
79420,79063273-1,NaN,PRINTING
79421,79074031-1,NaN,PRINTING


## Bước 5. UNION ALL

In [56]:
df_status = pd.concat ([ df_planning_pending, df_kitting_pending, df_printing_pending, df_BCSX_CRD[['SO#', 'Cell In', 'Status Order']] ], ignore_index = True)
df_status

,SO#,Cell In,Status Order
0,79173263-1,,PLANNING
1,79163865-1,,PLANNING
2,79163865-2,,PLANNING
3,79163865-3,,PLANNING
4,79163865-4,,PLANNING
...,...,...,...
82998,79329523-1,AS 72044,PACKING
82999,79329523-2,AS 72043,PACKING
83000,79329578-1,AS 13153,PACKING
83001,79329907-1,AS 72045,PACKING


In [97]:
#Check unique SO#, Cell In, Status
df_status.drop_duplicates(subset=['SO#', 'Cell In', 'Status Order'], inplace = True)
sr = df_status['SO#'].value_counts()
printing_SO = sr[sr==2].index

printing_SO_Status = df_status[df_status['SO#'].isin(printing_SO)].sort_values('SO#')
printing_SO_Status

,SO#,Cell In,Status Order
73921,71324345-1,NaN,PACKING
43012,71324345-1,NaN,KITTING
57733,73367792-1,NaN,KITTING
62283,73367792-1,AS 8632,PRINTING
62284,73655517-1,AS 8626,PRINTING
...,...,...,...
365,79340286-1,,PLANNING
1569,79347422-1,AS 73254,KITTING
450,79347422-1,,PLANNING
1579,79349462-1,NaN,KITTING


In [98]:
### DROP DUPLICATE + GIỮ LẠI STATUS GẦN NHẤT

#sort by our rule
priority_dict = {'PACKING':1, 'PRINTING':2, 'KITTING':3, 'PLANNING':4 }
df_status['priority'] = df_status['Status Order'].map(priority_dict)
df_status = df_status.sort_values(by=['SO#', 'priority'])

# Drop duplicate, giữ lại dòng đầu tiên
df_status.drop_duplicates(['SO#'], keep='first', inplace = True)
df_status

,SO#,Cell In,Status Order,priority
25253,59325787-1,NaN,KITTING,3
25254,59325787-2,NaN,KITTING,3
17674,60951313-2,NaN,KITTING,3
16924,60998495-1,NaN,KITTING,3
15808,61448392-1,NaN,KITTING,3
...,...,...,...,...
1504,79420420-1,,PLANNING,4
1443,79420431-1,,PLANNING,4
1427,79420663-1,,PLANNING,4
1461,79420789-1,,PLANNING,4


## Bước 6. INNER JOIN CRD

In [99]:
from datetime import date

In [100]:
df_production = pd.merge(df_status, df_CRD, \
                       on=['SO#','SO#'], how="inner") \
            [['SO#', 'Cell In', 'REQUEST_DATE', 'QTY', 'ITEM CODE', 'RBO', 'Status Order', 'TOTAL PRICE', 'MACHINE' ]]

#Request_Date có dtype: float64 => convert sang Date
#pd.to_datetime considers the float value is milliseconds since 1970.
#1899-12-30 = value 0 

df_production['REQUEST_DATE'] = pd.to_datetime(df_production['REQUEST_DATE'], unit = 'D', origin = '1899-12-30').dt.date
df_production ['Today'] = date.today()
df_production ['D'] = df_production['REQUEST_DATE'] - df_production ['Today']

df_production ['Cell In'] = np.where (df_production ['Cell In'].isnull() == True, 'IPPS', df_production ['Cell In'])


#Upper Column Names
df_production.rename(columns=str.upper, inplace = True)

df_production


### Code testing
#df_production [(df_production['STATUS ORDER'] == 'KITTING') & (df_production['CELL IN'] != 'IPPS')]
#df_production [df_production['STATUS ORDER'] == 'PLANNING']

,SO#,CELL IN,REQUEST_DATE,QTY,ITEM CODE,RBO,STATUS ORDER,TOTAL PRICE,MACHINE,TODAY,D
0,64293487-1,IPPS,2022-01-12,3402.0,B2582,ASDW,KITTING,57,MÁY 2471,2022-05-26,-134 days
1,64293487-2,IPPS,2022-01-12,3402.0,B2196,ASDW,KITTING,41,MÁY 2085,2022-05-26,-134 days
2,67524480-1,IPPS,2022-02-15,25.0,B16450,ASDW,PACKING,0,MÁY 16339,2022-05-26,-100 days
3,67524480-10,IPPS,2022-02-15,80.0,B16443,ASDW,PACKING,0,MÁY 16332,2022-05-26,-100 days
4,67524480-11,IPPS,2022-02-15,135.0,B16430,ASDW,PACKING,0,MÁY 16319,2022-05-26,-100 days
...,...,...,...,...,...,...,...,...,...,...,...
14594,79420420-1,,2022-05-14,841.0,B6670,ASDW,PLANNING,6,MÁY 6559,2022-05-26,-12 days
14595,79420431-1,,2022-05-12,30.0,B16906,ASDW,PLANNING,0,MÁY 16795,2022-05-26,-14 days
14596,79420663-1,,2022-05-17,12.0,B7749,ASDW,PLANNING,0,MÁY 7638,2022-05-26,-9 days
14597,79420789-1,,2022-05-17,10.0,B8955,ASDW,PLANNING,2877,MÁY 8844,2022-05-26,-9 days


## Bước 7. INVENTORY JOIN

In [ ]:
### Những SO# đã nhập kho (có trong dữ liệu Inventory) là những SO# đã hoàn thành
# 2 hướng xử lý: Mark COMPLETE hoặc ANTI JOIN. 
# Mark COMPLETE <-> khi định lấy tổng SO# trong CRD tách thành số SO# đã đưa vào sản xuất | sản xuất xong | chưa đưa vào sản xuất

In [101]:
df_inventory = dict_df .get('Inventory - 6')

In [102]:
# Hướng 2: ANTI JOIN

df_production_outer_inventory = pd.merge(df_production, df_inventory, \
                       on=['SO#','SO#'], how="outer", indicator=True)

df_production_outer_inventory = df_production_outer_inventory[df_production_outer_inventory['_merge'] == 'left_only'] [list (df_production)]
df_production_outer_inventory

,SO#,CELL IN,REQUEST_DATE,QTY,ITEM CODE,RBO,STATUS ORDER,TOTAL PRICE,MACHINE,TODAY,D
0,64293487-1,IPPS,2022-01-12,3402.0,B2582,ASDW,KITTING,57.0,MÁY 2471,2022-05-26,-134 days
1,64293487-2,IPPS,2022-01-12,3402.0,B2196,ASDW,KITTING,41.0,MÁY 2085,2022-05-26,-134 days
2,67524480-1,IPPS,2022-02-15,25.0,B16450,ASDW,PACKING,0.0,MÁY 16339,2022-05-26,-100 days
3,67524480-10,IPPS,2022-02-15,80.0,B16443,ASDW,PACKING,0.0,MÁY 16332,2022-05-26,-100 days
4,67524480-11,IPPS,2022-02-15,135.0,B16430,ASDW,PACKING,0.0,MÁY 16319,2022-05-26,-100 days
...,...,...,...,...,...,...,...,...,...,...,...
14594,79420420-1,,2022-05-14,841.0,B6670,ASDW,PLANNING,6.0,MÁY 6559,2022-05-26,-12 days
14595,79420431-1,,2022-05-12,30.0,B16906,ASDW,PLANNING,0.0,MÁY 16795,2022-05-26,-14 days
14596,79420663-1,,2022-05-17,12.0,B7749,ASDW,PLANNING,0.0,MÁY 7638,2022-05-26,-9 days
14597,79420789-1,,2022-05-17,10.0,B8955,ASDW,PLANNING,2877.0,MÁY 8844,2022-05-26,-9 days


## Bước 8. Xuất

In [61]:
df_production_outer_inventory.to_excel('FINAL.xlsx')